The objective of this demo is to showcase a scenario in which an hypothetical bank wants to build an AI system to support decision-making regarding credit concession to potential clients. The AI system should be trustable, hence they want to use TrustML to assess the trustworthiness of the candidate classification models before their deployment.

We use the Statlog dataset to simulate the scenario and the TrustML package to verify the model's trustworthiness.

# Step 1: Defining the configuration file
In this scenario, the company is interested in having an AI model complying with several trustworthiness criteria. In decreasing order of importance for the bank, the AI system should comply with:
1. Performance: It is worse to class a customer as good when they are bad, than it is to class a customer as bad when they are good.
2. Uncertainty: Knowing how uncertain the prediction is might influence the banker to (not) assume the risks.
3. Explainability: The bank will need models that provide explanations to the bankers, in order to enable the comprehension of how the AI system predicts the credit risk.
4. Fairness: Ethical aspects for the bank to prevent scandal. Protected attributes: gender, age.

Based on this criteria, we define an appropiate configuration file accounting for the trustworthiness definition and the weights of the metrics and associated dimensions. We will use the weighted average assessment method.

This is the content of the configuration file we will use:
```yaml
metrics:
    - AccuracySKL
    - PrecisionSKL:
        multiclass_average: "binary"
    - RecallSKL:
        multiclass_average: "binary"
    - PPercentageSKL:
        protected_attributes: [statussex_A91,statussex_A92,statussex_A93,statussex_A94]
        positive_class: 1
    - EqualOpportunitySKL:
        protected_attributes: [statussex_A91,statussex_A92,statussex_A93,statussex_A94]
        positive_class: 1
    - FaithfulnessLIMESKL:
        explainer_path: "demos/german_credit/lime_explainer"
    - InvertedExpectedCalibrationSKL
    - InvertedBrierSKL
assessment_method:
    WeightedAverage:
        performance-0.4:
            AccuracySKL: 0.1
            PrecisionSKL: 0.2
            RecallSKL: 0.7
        uncertainty-0.3:                  
            InvertedBrierSKL: 0.5
            InvertedExpectedCalibrationSKL: 0.5
        explainability-0.2:                
            FaithfulnessLIMESKL: 1
        fairness-0.1:             
            PPercentageSKL: 0.5
            EqualOpportunitySKL: 0.5
```

# Step 2: Import relevant packages
The first step will consist in importing the TrustML package, the classification model that we will use in the demo (RandomForestClassifier) and some supporting functions/modules, notably pandas for the dataset loading/manipulation, LimeTabularExplainer to train and evaluate an explainer for the model, and train_test_split to partition the dataset.

In [ ]:
from os import path
from dill import dump
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from lime.lime_tabular import LimeTabularExplainer
import pandas as pd
from trustML.computation import TrustComputation

We force pandas to print every column of the dataset when needed.

In [ ]:

pd.reset_option("max_columns")
pd.set_option('display.max_columns', None)

# Step 2: Preliminar operations
We define the dataset header, as it is not present in the actual dataset's CSV.

We also specify which is the target ('classification' column), and we classify each feature according to its type.

In [ ]:
demo_path = 'demos/german_credit/'
file_dataset = demo_path + 'german.data'
path_configuration_bank = demo_path + 'config_german_credit_banker_corporate.yml'

# DATASET HEADER
header = ['existingchecking', 'duration', 'credithistory', 'purpose', 'creditamount', 
         'savings', 'employmentsince', 'installmentrate', 'statussex', 'otherdebtors', 
         'residencesince', 'property', 'age', 'otherinstallmentplans', 'housing', 
         'existingcredits', 'job', 'peopleliable', 'telephone', 'foreignworker', 'classification']
label_column = 'classification'

numerical_variables = ['duration', 'creditamount', 'installmentrate', 'residencesince', 'age', 'existingcredits', 'peopleliable', 'classification']
categorical_variables = ['existingchecking', 'credithistory', 'purpose', 'savings', 'employmentsince',
           'statussex', 'otherdebtors', 'property', 'otherinstallmentplans', 'housing', 'job', 
           'telephone', 'foreignworker']

# protected_variables = ['statussex_A91', 'statussex_A92', 'statussex_A93', 'statussex_A94']

# Step 3: Load dataset and create train/test splits
Now we load the CSV file and perform some basic preprocessing, like replacing the target column's values from [1,2] -> [1,0] for interpretability purposes, and encoding the categorical features into dummy variables to ensure the classification model will be able to be trained on our dataset.

In [ ]:
# DATASET LOAD
pd_dataset = pd.read_csv(file_dataset, names=header, delimiter= ' ')
pd_dataset.head(10)

# CLASSIFICATION COLUMN. [1,2] -> [1,0]
pd_dataset.classification.replace([1,2], [1,0], inplace=True)
pd_dataset.classification.value_counts()

# LABEL ENCODING FOR CATEGORICAL/ORDINAL FEATURES
pd_dataset = pd.get_dummies(pd_dataset)
#print(pd_dataset.columns)

Now we will split the dataset into training and test, with a 70%-30% proportion.

In [ ]:
# TRAIN-TEST SPLIT
# SPLIT FEATURES-TARGET (X-Y)
data_x = pd_dataset.drop(label_column, axis=1)
data_y = pd_dataset[label_column]
# SPLIT DATA
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.3, random_state=2)

# Step 4: Create and train the classifier
We will now train a random forest classifier on the training set, as well as a lime tabular explainer that will be able to provide prediction's explanations based on the random forest's output. This explainer will be also assessed for trustworthiness as part of the explainability-related metrics.

In [ ]:
# TRAIN A RANDOM FOREST CLASSIFIER AND CREATE A TABULAR EXPLAINER
rf_classifier = RandomForestClassifier(max_depth=10, n_estimators=100,criterion='gini')
rf_model = rf_classifier.fit(train_x, train_y)
lime_explainer = LimeTabularExplainer(train_x.values, feature_names=data_x.columns.values, class_names=[1,0])

# Step 5: Store required assets
The explainability-related metric *FaithfulnessLIMESKL* requires an additional properties, a LIME tabular explainer. Therefore, we will store such asset into the directory we have set in the configuration file. The "explainer_path" property corresponds to the previously fitted lime tabular explainer. We store it using dill.

In [ ]:
# SAVE EXPLAINER TO THE DEMO DIRECTORY (OPTIONAL, ONLY NECESSARY IF IT IS NOT ALREADY PRESENT)
with open(demo_path + 'lime_explainer', 'wb') as explainer_file:
    dump(lime_explainer, explainer_file)

# Step 6: Compute the trustworthiness
Once trained, we will assess its trustworthiness with the TrustML package. For this, we instantiate a TrustComputation, we call the load_trust_definition method with the path to the configuration file we specified, and lastly we call the compute_trust function, passing the trained model and the test dataset (features and target) to evaluate the model's trustworthiness in such dataset. This function stores the trust assessment as a JSON-formatted string.

In [ ]:
trust_german_credit = TrustComputation()
trust_german_credit.load_trust_definition(config_path=path_configuration_bank)
trust_german_credit.compute_trust(trained_model=rf_model, data_x=test_x, data_y=test_y)

Now we can print the complete trustworthiness assessment as a JSON-formatted string using the getTrustworthinessScore function:

In [ ]:
print(trust_german_credit.get_trust_as_JSON())

```javascript
{
    "name": "Trust",
    "weighted_score": 0.75,
    "children": [{
            "name": "performance",
            "weight": 0.4,
            "weighted_score": 0.35,
            "raw_score": 0.89,
            "children": [{
                    "name": "AccuracySKL",
                    "weight": 0.1,
                    "weighted_score": 0.08,
                    "raw_score": 0.75
                },
                {
                    "name": "PrecisionSKL",
                    "weight": 0.2,
                    "weighted_score": 0.15,
                    "raw_score": 0.75
                },
                {
                    "name": "RecallSKL",
                    "weight": 0.7,
                    "weighted_score": 0.66,
                    "raw_score": 0.95
                }
            ]
        },
        {
            "name": "uncertainty",
            "weight": 0.3,
            "weighted_score": 0.2,
            "raw_score": 0.67,
            "children": [{
                    "name": "InvertedBrierSKL",
                    "weight": 0.5,
                    "weighted_score": 0.18,
                    "raw_score": 0.36
                },
                {
                    "name": "InvertedExpectedCalibrationSKL",
                    "weight": 0.5,
                    "weighted_score": 0.48,
                    "raw_score": 0.97
                }
            ]
        },
        {
            "name": "explainability",
            "weight": 0.2,
            "weighted_score": 0.11,
            "raw_score": 0.53,
            "children": [{
                "name": "FaithfulnessLIMESKL",
                "weight": 1,
                "weighted_score": 0.53,
                "raw_score": 0.53
            }]
        },
        {
            "name": "fairness",
            "weight": 0.1,
            "weighted_score": 0.09,
            "raw_score": 0.93,
            "children": [{
                    "name": "PPercentageSKL",
                    "weight": 0.5,
                    "weighted_score": 0.45,
                    "raw_score": 0.91
                },
                {
                    "name": "EqualOpportunitySKL",
                    "weight": 0.5,
                    "weighted_score": 0.48,
                    "raw_score": 0.96
                }
            ]
        }
    ]
}
```

# Conclusions
Which results in a value of 0.75 of the trustworthiness indicator for the classification model that will conform the AI system.

This notebook has illustrated how easy it is to use the TrustML package to evaluate the trustworthiness of a classification model intended to be used as part of an AI system. In this case, the TrustML package has been used as part of a model building pipeline, obtaining a trustworthiness assessment of 0.75 (out of 1). According to the trustability criteria of the ficticious company, the model would not deemed as acceptable, as their threshold is 0.9. Therefore, the model/training data would require modifications aiming to improve the overall trustworthiness of the model.